In [11]:
from bs4 import BeautifulSoup
# import time
import requests
import json
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor

In [12]:
#define variables

url = "https://letterboxd.com/films/popular/decade/2010s/page/"
page_count = 2
soup = 0
link = 0

In [13]:
#To parse the given page
def parse_page(url = url):
    options = Options()
    options.add_argument("--headless")
    service = Service(executable_path= r"C:\Users\abhyu\Downloads\geckodriver-v0.32.1-win64\geckodriver.exe")
    driver = webdriver.Firefox(options=options, service=service)

    driver.get(url)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    # print(soup.prettify)
    
    return soup

In [14]:
#data for film in individual page

def more_film_data(link=link):
    film_soup = parse_page(link)
    
    actor_data = film_soup.find_all('a', {"class" : "text-slug tooltip"}, limit= 3)
    actors = []
    for actor in actor_data:
        actors.append(actor.text)

    director = film_soup.find('span', {"class" : "prettify"}).text

    # country_or_countries = film_soup.find()
    country_tag = film_soup.find('h3', text=re.compile(r'Country|Countries')).find_next('div')
    country = [a.text for a in country_tag.find_all('a')]

    language = film_soup.find('h3', text=re.compile(r'Original Language|Language')).find_next('div').text.strip()

    rating = film_soup.find('meta', {"name" : "twitter:data2"})["content"]
    rating = float(rating.replace(' out of 5',''))

    letterboxd_views = film_soup.find('li', class_="stat filmstat-watches").find_next('a')['data-original-title']
    letterboxd_views = int(''.join(filter(str.isdigit, letterboxd_views)))

    #Genre

    #Number of films in the franchise

    #LINKS FOR OTHER SITES
    imdb_link = film_soup.find('a', {"data-track-action" : "IMDb"})['href']
    tmdb_link = film_soup.find('a', {"data-track-action" : "TMDb"})['href']

    more_film_list = [rating, letterboxd_views, director, country, language, actors, imdb_link, tmdb_link]
    # more_film_list.append(actors)

    
    return more_film_list

In [15]:
#Create a dataframe of all films in a given page
def collect_data(soup = soup):
    # ratings = soup.find_all('li', {"class" : "listitem poster-container"}) 
    list_of_movies = soup.find_all('div', {"data-component-class" : "globals.comps.FilmPosterComponent"}) 

    df_page = pd.DataFrame()
    

    #One movie at a time
    for index, attributes in enumerate(list_of_movies):
        # print(attributes.get('data-film-name')+', '+ attributes.get('data-film-release-year'))
        link = 'https://letterboxd.com' + attributes.get('data-film-link')
        row = [attributes.text, attributes.get('data-film-name'), attributes.get('data-film-release-year'), link]
        print(attributes.get('data-film-name'))

        more_film_list = more_film_data(link)
        row.extend(more_film_list)
        
        # print(row)
        df_row = pd.DataFrame(row)

        df_page = pd.concat([df_page, df_row], axis=1)

        #Testing
        if index == 3:
            break
    
    
    return df_page
        
    

In [16]:
# loop over the pages
def page_loop(url=url, page_count=page_count):

   
    for count in range(1, page_count):
        url = url + str(count)
        soup = parse_page(url)
        df_page = collect_data(soup)
        df = pd.concat([df, df_page])
        # print(count)
    return df.transpose()
        

In [17]:
# Alternative

# def scrape_website(url, page_count):
#     # Initialize the dataframe and the list of urls to be scraped
#     df = pd.DataFrame()
#     urls = [url + str(i) for i in range(1, page_count + 1)]

#     # Define the maximum number of threads to be used
#     max_threads = 10

#     # Initialize a list to store the futures
#     futures = []

#     # Define the function to be executed by each thread
#     def thread_fn(url):
#         soup = parse_page(url)
#         return collect_data(soup)

#     # Use ThreadPoolExecutor to execute the function for each url in parallel
#     with ThreadPoolExecutor(max_workers=max_threads) as executor:
#         for url in urls:
#             futures.append(executor.submit(thread_fn, url))

#     # Concatenate the dataframes from each thread into a single dataframe
#     for future in futures:
#         df_page = future.result()
#         df = pd.concat([df, df_page])

#     return df.transpose()

In [18]:
def get_data(url=url, page_count=page_count):
    df = page_loop(url, page_count)
    # df.columns = ["Name and Year", "Name", "Year", "Link", "Letterboxd Rating", "Letterboxd Views", "Director", "Country", "Language", "Actors" ]
    return df

In [19]:
df = get_data(url, page_count+1)
df

""


In [20]:
# film_soup = parse_page("https://letterboxd.com/film/joker-2019/")